## Hyperparameter optimization

In [ ]:
import optuna


def back_transform(x, max_train):
    xexp = np.exp(x)
    return max_train + 1 - xexp


early_stop = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0,
    patience=5,
    verbose=0,
    mode="auto",
    baseline=None,
    restore_best_weights=True,
)


reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', factor=0.1, patience=2, verbose=0,
    mode='auto', min_delta=0.0001, cooldown=0, min_lr=0.00001
)


def objective(trial: optuna.trial.Trial):
    lambda_ = trial.suggest_float("lambda_", 22.0, 28.0)
    lr = trial.suggest_loguniform("lr", 1e-5, 0.001)
    dropout_rate_l1 = trial.suggest_uniform("dropout_rate_l1", 0.0, 0.5)
    dropout_rate_l2 = trial.suggest_uniform("dropout_rate_l2", 0.0, 0.5)
    layer_size = trial.suggest_categorical("layer_size", 
                                           ["32+16", "64+32", 
                                            "128+64", "128+128"],
                                          )
    layer_size = tuple([int(ls) for ls in layer_size.split("+")])
    seed = trial.suggest_int("seed", 162, 9999999)
    np.random.seed(seed)
    # Split train from test data
    x_ind = [*range(train_x.shape[0])]
    np.random.shuffle(x_ind)
    train_x_trial, val_x_trial = train_x[:420000, :].copy(), train_x[420000:, :].copy()
    train_y_trial, val_y_trial = train_y[:420000].copy(), train_y[420000:].copy()
    max_train_out = np.max(train_y_trial)
    # Transform
    train_y_trial_transformed = reflect_and_log(train_y_trial, max_train_out)
    val_y_trial_transformed = reflect_and_log(val_y_trial, max_train_out)
    # Instantiate model
    model = PivenKerasRegressor(build_fn=piven_model, 
                              input_dim=train_x_trial.shape[-1], 
                              dense_units=layer_size, 
                              dropout_rate=(dropout_rate_l1, dropout_rate_l2),
                              lambda_=lambda_,
                              lr=lr)
    # Put model in pipeline
    pipeline = Pipeline([
        ("preprocess", StandardScaler()),
        ("model", model)
    ])
    model_ttr = PivenTransformedTargetRegressor(regressor=pipeline,
                                                transformer=StandardScaler())
    # Fit
    model_ttr.fit(train_x_trial, train_y_trial_transformed, model__epochs=25, 
                  model__validation_split=0.1, model__batch_size=64,
                  model__verbose=False, model__callbacks=[early_stop, reduce_lr])
    y_pred, y_pred_pi_low, y_pred_pi_high = model_ttr.predict(val_x_trial, return_prediction_intervals=True)
    # Back-transform the predictions and PIs
    y_pred = back_transform(y_pred, max_train_out)
    y_pi_low = back_transform(y_pred_pi_high, max_train_out)
    y_pi_high = back_transform(y_pred_pi_low, max_train_out)
    # Compute metrics
    cov_trial = coverage(val_y_trial, y_pi_low, y_pi_high)
    piw_trial = pi_width(y_pi_low, y_pi_high)
    loss_trial = piven_loss_numpy(val_y_trial, y_pred, y_pi_low, y_pi_high, lambda_, 160.0, 0.05)
    # Set metrics
    trial.set_user_attr("coverage", float(cov_trial))
    trial.set_user_attr("pi_width", float(piw_trial))
    # Return model loss
    return loss_trial

In [ ]:
experiment = optuna.create_study(direction="minimize",
                                 study_name="piven_year_msd",
                                 storage='sqlite:///data/experiment_results.db',
                                 load_if_exists=True)

In [ ]:
experiment.optimize(objective, n_trials=50)